In [1]:
import pandas as pd
import requests
import json
import os
import datetime
from getpass import getpass

In [2]:
url = 'https://natthawat.live/api'

# Set cookies and userId

In [3]:
username = input('Username: ')
password = getpass('Password: ')
login = {'username': username, 'password': password}
response = requests.post('%s/km/auth/login' % url,data=login)

if response.status_code == 201 or response.status_code == 200:
    cookies = response.cookies.get_dict()
    userId = json.loads(response.text)['id']
else:
    print("Username or password is not correct")

Username: root
Password: ········


In [4]:
# Debug check
debug = False
if debug:
    print(cookies)
    print(userId)

# Declare Function to Get Questions From Backend API
Return dataframe that contains
```
column: ['id', 'question', 'answer', 'updatedDate', 'subcategory', 'category', 'category_id',
       'subcategory_id', 'lastEditor_id', 'lastEditor_name','lastEditor_username']
```

In [25]:
def getQuestionsFromBackendAPI():
    response = requests.get('%s/km/faq' % url)
    faqs = json.loads(response.text)

    # Convert from all faqs to df
    df=pd.DataFrame(faqs)

    # Spread object inside each row
    # category: {id, category} -> category_id, category
    df['category_id'] = df['category'].apply(lambda x: x['id'])
    df['category'] = df['category'].apply(lambda x: x['category'])
    
    # subcategory: {id, subcategory} -> category_id, subcategory
    df['subcategory_id'] = df['subcategory'].apply(lambda x: x['id'])
    df['subcategory'] = df['subcategory'].apply(lambda x: x['subcategory'])

    # lastEditor: {id, name, username} -> lastEditor_id, lastEditor_name, lastEditor_username
    df['lastEditor_id'] = df['lastEditor'].apply(lambda x: x['id'])
    df['lastEditor_name'] = df['lastEditor'].apply(lambda x: x['name'])
    df['lastEditor_username'] = df['lastEditor'].apply(lambda x: x['username'])
    
    # Remove lastEditor
    df = df.drop(columns="lastEditor")

    return df

In [26]:
getQuestionsFromBackendAPI()

,id,question,answer,updatedDate,subcategory,category,category_id,subcategory_id,lastEditor_id,lastEditor_name,lastEditor_username
0,01ac6c58-e185-4884-af99-6bad331b41bd,สามารถลงทะเบียนเกินหน่วยกิตที่หลักสูตรกำหนดได้...,1. นักศึกษาปกติ ชั้นปีที่ 1 ถึง ชั้นปีที่ 4 เท...,2021-05-12T19:43:13.000Z,หลักสูตร,ลงทะเบียนเรียน,92302aa2-bf0f-4ee6-9288-ada605f982d0,63138d4c-450c-4821-8ece-cf881af8484a,root-id,rootadmin,root
1,03528d65-e4f0-49e3-bdbd-3bdd1758d0bc,วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลั...,วิชาภาคบังคับที่มีวิชาต่อเนื่อง แยกตามกลุ่มวิช...,2021-05-12T19:42:58.000Z,หลักสูตรปกติ,หลักสูตร,057c3235-0d2b-4d9b-ac64-d736401c66d1,919dae89-bf87-4777-aef1-918fc0a4e862,root-id,rootadmin,root
2,03d502b3-967b-4e07-8177-bd461ed95dc5,ใบแจ้งชำระเงินค่าลงทะเบียน สั่งพิมพ์ได้จากที่ไหน,Login เข้าระบบ https://sinfo.kmutt.ac.th/ เลื...,2021-05-12T19:43:11.000Z,เอกสาร,ลงทะเบียนเรียน,92302aa2-bf0f-4ee6-9288-ada605f982d0,fa40cb26-386c-4330-a166-f26f41f7f0f7,root-id,rootadmin,root
3,09192e6e-2bc1-49a3-9ac0-dc01b5982885,ตอนดรอปต้องขอลายเซ็นใครบ้าง?,หากนักศึกษาต้องการถอดถอนวิชาหลังจาก 2 สัปดาห์แ...,2021-05-12T19:43:08.000Z,เพิ่มลดรายวิชา,ลงทะเบียนเรียน,92302aa2-bf0f-4ee6-9288-ada605f982d0,a52f4ea3-2ec2-4373-9351-1cae1be707a7,root-id,rootadmin,root
4,09d2cce4-70dd-45fa-9e94-da3d71d3c383,ขอทรานสคริปต์ออนไลน์ได้ที่ไหน,นักศึกษาสามารถทำตามรายละเอียดได้ตามนี้\n1. ดาว...,2021-05-12T19:43:12.000Z,เอกสาร,ลงทะเบียนเรียน,92302aa2-bf0f-4ee6-9288-ada605f982d0,fa40cb26-386c-4330-a166-f26f41f7f0f7,root-id,rootadmin,root
...,...,...,...,...,...,...,...,...,...,...,...
166,f6138050-e917-43b1-92aa-3ff5a5290c12,วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่1-2 เ...,วิชาวิศวกรรมเบื้องต้นและพื้นฐานคอมพิวเตอร์ เช่...,2021-05-12T19:43:00.000Z,หลักสูตรนานาชาติ,หลักสูตร,057c3235-0d2b-4d9b-ac64-d736401c66d1,4a51d8ad-29b1-4d04-9f9c-09f6b83ab84e,root-id,rootadmin,root
167,f61bacee-e3e0-45e6-94bd-4725d82ed8c0,วิชา CPE122/Circuits มีวิชาตัวต่อมั้ยครับ?,ไม่มีค่ะ,2021-05-12T19:43:04.000Z,คำถามทั่วไป,หลักสูตร,057c3235-0d2b-4d9b-ac64-d736401c66d1,bcbe6b15-6753-4429-9b95-7ecaf7e8c329,root-id,rootadmin,root
168,f648935f-9e93-44ff-a42a-d0a0338a2618,จะต้องใช้เอกสารอะไรบ้างในการสมัครเข้าศึกษา,เอกสารและรายละเอียดขึ้นอยู่การสมัครของแต่ล่ะโค...,2021-05-12T19:42:51.000Z,เอกสาร,การรับเข้านักศึกษา,5008a257-f9dd-41a1-8c7e-d40842def6ce,bcd0db00-6bad-4fe2-8e60-4a30fc7edd79,root-id,rootadmin,root
169,fc7996c8-3f0a-42e2-a620-5dd987865c45,วิศวะคอมหลักสูตรปกติมีอัตราค่าเรียนเท่าไหร่,"รวมค่าใช้จ่ายตลอดหลักสูตร 4 ปี ประมาณ 168,000 ...",2021-05-12T19:43:02.000Z,หลักสูตรปกติ,หลักสูตร,057c3235-0d2b-4d9b-ac64-d736401c66d1,919dae89-bf87-4777-aef1-918fc0a4e862,root-id,rootadmin,root


# Export Questions from Backendend API to CSV

In [27]:
# Get all questions from backend API
questions_df = getQuestionsFromBackendAPI()

# Set file path
path='csv/'

# Try create folder directory
try:
    # Create target Directory
    os.mkdir(path)
except FileExistsError:
    pass

# Set current time for time file
date_now = datetime.datetime.now()
date_now_str = str(date_now.year)+"-"+\
                str(date_now.month)+"-"+\
                str(date_now.day)+"-"+\
                str(date_now.hour)+"-"+\
                str(date_now.minute)+"-"+\
                str(date_now.second)

# Export questions from km to csv
# File format: backup-km-yyyy-mm-dd-hh-mm-ss.csv
questions_df.to_csv(path_or_buf = path + "backup_km" + date_now_str + ".csv", index=False)